In [ ]:
ls

In [ ]:
from matplotlib import pyplot as plt
from scipy.stats import multivariate_normal, cumfreq
import numpy as np


In [ ]:
from Defines import ParticleInHepMC, DMAnihilationCompoundParticle
from ReadTable import read_pppc4, read_cosmiXs

In [ ]:
from Get_MG_Data import Files


# $\bar{p},\bar{n}$能谱

In [ ]:
list_all_pbar = []
list_all_nbar = []
EventsNumber = 0
for f in Files:
    list_all_pbar += f.get_pbar()
    list_all_nbar += f.get_nbar()
    EventsNumber += f.events_number

list_all_antinucleons = list_all_pbar + list_all_nbar

## 动能分布

In [ ]:
# 将antinuclon中每个元素的动能提取出来组成一个numpy数组
T_antinucleon = np.array([i.T for i in list_all_antinucleons])
N_T_antinucleon, bins_T_antinucleon = np.histogram(T_antinucleon, bins='doane', density=False)
dNdT_T_antinucleon = N_T_antinucleon / np.diff(bins_T_antinucleon) / EventsNumber

T_pbar = np.array([i.T for i in list_all_pbar])
N_T_pbar, bins_T_pbar = np.histogram(T_pbar, bins='doane', density=False)
dNdT_T_pbar = N_T_pbar / np.diff(bins_T_pbar) / EventsNumber

T_nbar = np.array([i.T for i in list_all_nbar])
N_T_nbar, bins_T_nbar = np.histogram(T_nbar, bins='doane', density=False)
dNdT_T_nbar = N_T_nbar / np.diff(bins_T_nbar) / EventsNumber


In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10,8), sharex=True)
# plt.figure("反核子动能分布")

plot_bins_T_antinucleon = bins_T_antinucleon[:-1]+np.diff(bins_T_antinucleon)/2
ax[0].scatter(plot_bins_T_antinucleon, dNdT_T_antinucleon, s=5, label=r'$\bar{p}+\bar{n}$')
plot_bins_T_pbar = bins_T_pbar[:-1]+np.diff(bins_T_pbar)/2
ax[0].scatter(plot_bins_T_pbar, dNdT_T_pbar, s=5, marker=',', label=r'$\bar{p}$')
plot_bins_T_nbar = bins_T_nbar[:-1]+np.diff(bins_T_nbar)/2
ax[0].scatter(plot_bins_T_nbar, dNdT_T_nbar, s=5, marker='v', label=r'$\bar{n}$')

ax[0].set_yscale('log')
ax[0].set_ylabel(r'$\frac{dN}{dT}$ /(/GeV/annihilation)')
ax[0].legend(loc = "lower left")
ax[0].set_title('antinucleon kinetic energy distribution')

ax[1].scatter(plot_bins_T_antinucleon, dNdT_T_antinucleon*plot_bins_T_antinucleon**2.0, s=5, label=r'$\bar{p}+\bar{n}$')
ax[1].set_xscale('log')
ax[1].set_yscale('log')
ax[1].set_xlabel('T/GeV')
ax[1].set_ylabel(r'$\frac{dN}{dT}T^{2.7}$ /($GeV^{1.7}$/annihilation)')


# plt.xlim(1e0, 1e2)
# plt.ylim(1e-2, 10)

## x分布

In [ ]:
lg_x_pbar = np.array([i.lgx for i in list_all_antinucleons])
N_lgx_pbar, bins_lgx_pbar = np.histogram(lg_x_pbar, bins='doane', density=False)
dNdlgx_lgx_pbar = N_lgx_pbar / np.diff(bins_lgx_pbar) / EventsNumber

bins_x_pbar = 10**bins_lgx_pbar

x_pppc4, y_pppc4 = read_pppc4("../TestData/PPPC4/AtProduction_antiprotons.dat", 100, 'b')[:2]

In [ ]:
plt.figure("反核子x分布")

plt.scatter(bins_x_pbar[:-1]+np.diff(bins_x_pbar)/2, dNdlgx_lgx_pbar, s=10, label=r'$\bar{p}+\bar{n}$')
plt.bar(bins_x_pbar[:-1], height=dNdlgx_lgx_pbar, width=np.diff(bins_x_pbar), alpha=0.5, align='edge')
plt.plot(x_pppc4, y_pppc4, label='PPPC4')
plt.xlim(1e-5, 1e0)
plt.ylim(1e-2, 1e1)
plt.xscale('log')
plt.yscale('log')
plt.ylabel(r'$\frac{dN}{d logx}$ /(/annihilation)')
plt.xlabel('x=$K/D_{DM}$')
plt.legend()
plt.title(r"$\bar{p}+\bar{n}$ x primary spectra ($M_{DM}$="+f"{round(list_all_pbar[0].mass_DM)}GeV)")

# $\bar{D}$

In [ ]:
M_antideuteron = 1.875612928  # 反氘核质量，单位GeV
Pcoal = 0.195  # 聚结动量，单位GeV
r_Dbar = 3e-12 # 反氘核半径3fm，单位mm

x_Dbar_pppc4, dNdlgx_Dbar_pppc4, lgx_Dbar_pppc4 = read_pppc4("../TestData/PPPC4/AtProduction_antideuterons.dat", 100, 'b')

## Models


In [ ]:
def LookForAntideuteronInPhraseSpace(pbars:list[ParticleInHepMC], nbars:list[ParticleInHepMC], *, model:int=1, delta_r:float=0, sigma:float=1.8, delta:float=1.8) -> list[DMAnihilationCompoundParticle]:
    Dbars = []
    # delta *= 5.068 # fm - > GeV^-1
    MultiGauss = multivariate_normal(mean=[0, 0], cov=[sigma ** 2, 1/delta**2])
    for a_pbar in pbars:
        if a_pbar.IsComponent:
            continue
        for a_nbar in nbars:
            if a_nbar.IsComponent:
                continue
            delta_p =a_pbar.ComputeP3Distance(a_nbar)
            if (model == 1 or model == 2) and (delta_p <= Pcoal):
                Dbar = DMAnihilationCompoundParticle(0, a_pbar.P+a_nbar.P, M_antideuteron)
                Dbar.composition = [a_pbar, a_nbar]
                Dbar.A = 2
                Dbar.Z = -1
                Dbars.append(Dbar)
                a_pbar.IsComponent = True
                a_nbar.IsComponent = True
                break
            elif (model == 3) and (np.random.rand() <= 1-MultiGauss.cdf([delta_r, delta_p * 5.068/2], lower_limit=[0,0])*4):
                Dbar = DMAnihilationCompoundParticle(0, a_pbar.P+a_nbar.P, M_antideuteron)
                Dbar.composition = [a_pbar, a_nbar]
                Dbar.A = 2
                Dbar.Z = -1
                Dbars.append(Dbar)
                a_pbar.IsComponent = True
                a_nbar.IsComponent = True
                break
    return Dbars

### Model 1--Simple coalescence model
$$\Delta \vec{p}=|\vec{p}_{\bar{p}}-\vec{p}_{\bar{n}}|<p_{coal}$$

In [ ]:
list_Dbar_1 = []
for f in Files:
    f.reset_component()
    for event in f.events:
        list_pbar = event.get_pbar()
        list_nbar = event.get_nbar()
        list_Dbar_1 += LookForAntideuteronInPhraseSpace(list_pbar, list_nbar, model=1)

#### 累积频数

In [ ]:
lgx_Dbar1 = [i.lgx for i in list_Dbar_1]
# N_lgx_Dbar1, bins_lgx_Dbar1 = np.histogram(lgx_Dbar1, bins="doane")
res_Dbar1 = cumfreq(lgx_Dbar1, numbins=7)

plt.figure("反氘x累积分布")
bar_bins = res_Dbar1.lowerlimit+np.arange(res_Dbar1.cumcount.size)*res_Dbar1.binsize
plt.bar(bar_bins, res_Dbar1.cumcount/EventsNumber, width=res_Dbar1.binsize, align='edge', alpha=0.5)
plt.yscale("log")



### Model 2--Simple coalescence model with a sharp cutoff in distance
$$\begin{align*}
\Delta \vec{p}=|\vec{p}_{\bar{p}}-\vec{p}_{\bar{n}}|<p_{coal}\\
\Delta \vec{r}<3fm
\end{align*}$$


In [ ]:
list_Dbar_2 = []
for f in Files:
    f.reset_component()
    for event in f.events:
        vertices = event.vertices
        for i in range(length_ver:=len(vertices)):
            list_Dbar_2 += LookForAntideuteronInPhraseSpace(vertices[i].pbar, vertices[i].nbar, model=2)
            for j in range(i+1, length_ver):
                if (vertices[i].ComputeDistance(vertices[j]) <= r_Dbar):
                    list_Dbar_2 += LookForAntideuteronInPhraseSpace(vertices[i].pbar, vertices[j].nbar, model=2)
                    list_Dbar_2 += LookForAntideuteronInPhraseSpace(vertices[j].pbar, vertices[i].nbar, model=2)

In [ ]:
[i.x for i in list_Dbar_2]

### Model 3--Wigner approach with a Gaussian wavefunction (Gauss Wigner)
$$
\begin{align}
D(\Delta \vec{r}=\vec{r}_p-\vec{r}_n,\Delta \vec{p}=(\vec{p}_p-\vec{p}_n)/2)&=(\frac{\delta}{\sigma})^3 e^{-\Delta \vec{r}^2/(2\sigma^2)}e^{-\Delta \vec{p}^2\delta^2/2},\\
&?=4\cdot N([0,0],[\sigma^2,1/\delta^2])
\end{align}
$$
对每一对$\bar{p},\bar{n}$，对应有一个$D(\Delta \vec{r},\Delta \vec{p})$，表示聚结概率密度，我们可以取一个随机数，小于其则形成反氘，否则不能形成。

In [ ]:
list_Dbar_3 = []
for f in Files:
    f.reset_component()
    for event in f.events:
        vertices = event.vertices
        for i in range(len_ver:=len(vertices)):
            list_Dbar_3 += LookForAntideuteronInPhraseSpace(vertices[i].pbar, vertices[i].nbar, model=3)
            for j in range(i+1, len_ver):
                delta_r = vertices[i].ComputeDistance(vertices[j]) * 1e12 # mm->fm
                list_Dbar_3 += LookForAntideuteronInPhraseSpace(vertices[i].pbar, vertices[j].nbar, model=3, delta_r=delta_r)
                list_Dbar_3 += LookForAntideuteronInPhraseSpace(vertices[j].pbar, vertices[i].nbar, model=3, delta_r=delta_r)


In [ ]:
a = 1
# list_Dbar_3[a].composition[0].ComputeP3Distance(list_Dbar_3[a].composition[1])
list_Dbar_3[a].composition
print(f"==>> list_Dbar_3[0].composition: {list_Dbar_3[0].composition}")
print(f"==>> list_Dbar_3[1].composition: {list_Dbar_3[1].composition}")

#### 动能

In [ ]:
T_Dbar = np.array([i.T for i in list_Dbar_3])
N_T_Dbar, bins_T_Dbar = np.histogram(T_Dbar, bins='doane', density=False)
dNdT_T_Dbar = N_T_Dbar / np.diff(bins_T_Dbar) / EventsNumber

In [ ]:
plt.figure("反氘动能分布")

plt.scatter(bins_T_Dbar[:-1]+np.diff(bins_T_Dbar)/2, dNdT_T_Dbar, s=5, label=r'$\bar{D}$')

# plt.xlim(1e0, 1e2)
# plt.ylim(1e-2, 10)
plt.xscale('log')
plt.yscale('log')
plt.ylabel(r'$\frac{dN}{dT}$ /(/GeV/annihilation)')
plt.xlabel('T/GeV')
plt.legend(loc = "lower left")
plt.title('antinucleon kinetic energy distribution')

#### x

In [ ]:
lgx_Dbar = np.array([i.lgx for i in list_Dbar_3])
N_lgx_Dbar, bins_lgx_Dbar = np.histogram(lgx_Dbar, bins='doane', density=False)
dNdlgx_lgx_Dbar = N_lgx_Dbar / np.diff(bins_lgx_Dbar) / EventsNumber

bins_x_Dbar = 10**bins_lgx_Dbar

x_Dbar_cosmiXs, dNdlgx_Dbar_cosmiXs = read_cosmiXs("../TestData/AntiDeuterons/AtProduction-AntiD-GWF.dat", 100, 'bb')

In [ ]:
plt.figure("反氘x分布")

plt.scatter(bins_x_Dbar[:-1]+np.diff(bins_x_Dbar)/2, dNdlgx_lgx_Dbar, s=5, label=r'$\bar{D}$')
plt.plot(x_Dbar_pppc4, dNdlgx_Dbar_pppc4, label='PPPC4')
plt.scatter(x_Dbar_cosmiXs, dNdlgx_Dbar_cosmiXs, s=3, label='CosmiXs')
plt.xlim(1e-5, 1e0)
# plt.ylim(1e-2, 1e1)
plt.xscale('log')
plt.yscale('log')
plt.ylabel(r'$\frac{dN}{d logx}$ /(/annihilation)')
plt.xlabel('x=$K/D_{DM}$')
plt.legend()
plt.title(r"$\bar{D}$ x primary spectra ($M_{DM}$="+f"{round(list_all_pbar[0].mass_DM)}GeV)")

### Model 4--Wigner approach with Argonne function (Argonne Wigner)


### Model 5--Spherical approach (INCORRECT!)

## 动能分布